<div>
<img src="figures/svtLogo.png"/>
</div>

<center><h1>Mathematical Optimization for Engineers</h1></center>
<center><h2>Lab 3 - Gradient descent</h2></center>

In this lab, we will implement: 

1. The steepest descent method with Armijo and Wolfe linesearch algorithms<br>
<br>
2. Newton's method

We will use the [autograd](https://github.com/HIPS/autograd) package by Prof. Ryan Adams' research group to calculate the gradient and Hessian of our objective function. <i>autograd</i> uses automatic differentiation.

In [1]:
import sys
!{sys.executable} -m pip install numpy autograd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 kB 10.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=1d0dbb443fadb04d081e03bdf55c90169fc11d79d5fb7908efbb695259ba5f39
  Stored in directory: /home/jovyan/.cache/pip/wheels/22/73/06/557dc4f4ef68179b9d763930d6eec26b88ed7c389b19588a1c
Successfully built future


In [2]:
import numpy as np

# to calculate the inverse of the Hessian in Newton's method
from numpy.linalg import inv

# to calculate the gradient and Hessian of the objective function
from autograd import grad
from autograd import hessian

In [4]:
def objective_function(x): 
    f = 0.5 * (x[0] ** 2 + 20 * x[1] ** 2)
    return f

In [5]:
def rosenbrock(x):
    return ((x[0]-1)**2 + 100*(x[1]-x[0]**2)**2)

In [6]:
def polynomial(x): 
    return - 1.0 / 6 * x[0]**6 + 1 / 4 *x[0]**4 + 2 * x[0]**2

In [7]:
def gradient_function(func, x): 
    return grad(func)(x)

In [8]:
def hessian_function(func,x):
    return hessian(func)(x)

In [9]:
# input: (objective function, initial guess, step rule, c1, c2, initial alpha, optimality tolerance)
def steepest_descent(function, x0, stepRule, c1, c2, alpha0, tol=0.01):     
    
    xCur = x0
    fcur = function(xCur)
    gCur = gradient_function(function,xCur)
    
    # norm of the gradient at initial point for optimality condition
    nmg0 = np.linalg.norm(gradient_function(function, x0))
    
    # count number of steps taken by method
    it = 0
    
    # accumulate number of iterations needed by linesearch algorithm in every step
    ls_iter = 0
    
    # store iterates for plotting
    xIter=[]
    xIter.append(x0)
    
    while(np.linalg.norm(gCur)>tol*min(1,nmg0)): 

        it=it+1
        
        # calculate descent direction
        direction = -gCur
        
        # calculate step-length
        if (stepRule=='armijo'):
            alpha, ls_ = armijo(function,xCur,direction, c1, alpha0)
        elif (stepRule=='wolfe'):
            alpha, ls_ = wolfe(function,xCur,direction, c1, c2, alpha0)
        else:
            alpha = 0.01
        ls_iter = ls_iter + ls_
        
        # update current point
        
        xCur = xCur + alpha*direction
        fCur = function(xCur)
        gCur = gradient_function(function,xCur)
        
        xIter.append(xCur)

    return xCur, fcur, it, xIter, ls_iter

In [10]:
# input: (objective function, initial guess, optimality tolerance)
def newton_descent(function, x0, stepRule='no', tol=0.01): 

    xCur = x0
    fcur = function(xCur)
    gCur = gradient_function(function,xCur)
    hCur = hessian_function(function,xCur)    
    
    # norm of the gradient at initial point for optimality condition
    nmg0 = np.linalg.norm(gradient_function(function, x0))    
    
    # count number of steps taken by method    
    it = 0
    
    # store iterates for plotting    
    xIter=[]
    xIter.append(x0)
    
    while(np.linalg.norm(gCur)>tol*min(1,nmg0)):
        
        it=it+1
        
        # calculate descent direction
        direction = -1.0 * np.dot(inv(hCur),gCur)
        
        # calculate step-length or use natural length of 1
        if (stepRule ==  'no'):
            alpha = 1
        else:
            alpha = 0.01
        
        # update current point
        xCur = xCur + alpha * direction
        fCur = function(xCur)
        gCur = gradient_function(function,xCur)
        hCur = hessian_function(function,xCur)
        
        xIter.append(xCur)

    return xCur, fcur, it, xIter

In [11]:
def armijo(function, xcur, searchdirection, c1, alpha0): 
    #f(x+alpha p) \leq f(x) + c1 alpha grad(f).p
    
    alpha = alpha0
    xnew = xcur + alpha * searchdirection
    fcur = function(xcur)
    fnew = function(xnew)
    gradientCur = gradient_function(function,xcur)
    numiter = 0
    # check for Armijo condition 
    while (function(xnew) > function(xcur) + alpha*c1*np.dot(gradientCur,searchdirection)): 
        numiter += 1
        alpha = alpha/2.0
        xnew = xcur + alpha * searchdirection
        fnew = function(xnew)

    return alpha, numiter

In [12]:
def wolfe(function, xcur, searchdirection, c1, c2, alpha0): 
    #f(x+alpha p) \leq f(x) + c1 alpha grad(f)Tp
    #grad(f(x+alpha p))Tp \geq c2 grad(f)Tp
    
    alpha = alpha0
    xnew = xcur + alpha * searchdirection
    fcur = function(xcur)
    fnew = function(xnew)
    gradientCur = gradient_function(function,xcur)
    gradientNew = gradient_function(function,xnew)
    numiter = 0
    
    lb = 0
    ub = np.inf 
    
    # check for Wolfe conditions
    # adapted from https://sites.math.washington.edu/~burke/crs/408/notes/nlp/line.pdf (pg. 8)
    while True: 
        numiter += 1
        if ((fnew) > (fcur + c1 * alpha * np.dot(gradientCur,searchdirection))):
            ub = alpha
            alpha = 0.5 * (lb + ub)
        elif (np.dot(gradientNew,searchdirection) < c2 * np.dot(gradientCur,searchdirection)):
            lb = alpha
            if np.isinf(ub):
                alpha = 2 * lb
            else:
                alpha = 0.5 * (lb + ub)
        else:
            break
        xnew = xcur + alpha * searchdirection
        fnew = function(xnew)
        gradientNew = gradient_function(function,xnew)

    return alpha, numiter

## Solving quadratic unconstrained problem
###  Steepest descent

In [13]:
x0 = np.array([0.9, 0.1])
c1 = 0.5 
c2 = 0.6 
alpha0 = 0.125 
tol = 0.0001

print ("{:<40} {:^20} {:^20} {:^15} {:^15}".format('Method','xopt','fopt','iter','ls_iter'))

# using steepest descent with Armijo condition for linesearch
xopt, fopt, it, xIter, ls_iter = steepest_descent(objective_function, x0, 'armijo', c1, c2, alpha0)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Armijo with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")

xopt, fopt, it, xIter, ls_iter = steepest_descent(objective_function, x0, 'wolfe', c1, c2, alpha0)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Wolfe with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")

xopt, fopt, it, xIter, ls_iter = steepest_descent(objective_function, x0, 'armijo', c1, c2, alpha0, tol)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Armijo with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")


xopt, fopt, it, xIter, ls_iter = steepest_descent(objective_function, x0, 'wolfe', c1, c2, alpha0, tol)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Wolfe with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it,ls_iter))

Method                                           xopt                 fopt              iter           ls_iter    
Armijo with default tol = 0.01           [0.0069, -0.0001]          5.0500e-01           44              15             


Wolfe with default tol = 0.01            [0.0069, -0.0002]          5.0500e-01           34              62             


Armijo with tol = 0.0001                 [0.0001, -0.0000]          5.0500e-01           82              26             


Wolfe with tol = 0.0001                  [0.0001, -0.0000]          5.0500e-01           64              114            


### Newton's method

In [14]:
x0 = np.array([0.9, 0.1])
c1 = 0.5 
c2 = 0.6 
alpha0 = 0.125 
tol = 0.0001

print ("{:<40} {:^20} {:^20} {:^15}".format('Method','xopt','fopt','iter'))

xopt, fopt, it, xIter = newton_descent(objective_function, x0)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it))
print("\n")

xopt, fopt, it, xIter = newton_descent(objective_function, x0, tol=0.0001)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it))

Method                                           xopt                 fopt              iter      
Newton with default tol = 0.01           [0.0000, 0.0000]          5.0500e-01           1              


Newton with tol = 0.0001                 [0.0000, 0.0000]          5.0500e-01           1              


## Solving Rosenbrock's function
Try also different starting points and play around with the tuning factors.

### Steepest descent

In [15]:
x0 = np.array([0.9, 0.1])
c1 = 0.5 
c2 = 0.6 
alpha0 = 0.125 
tol = 0.0001

print ("{:<40} {:^20} {:^20} {:^15} {:^15}".format('Method','xopt','fopt','iter','ls_iter'))

# using steepest descent with Armijo condition for linesearch
xopt, fopt, it, xIter, ls_iter = steepest_descent(rosenbrock, x0, 'armijo', c1, c2, alpha0)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Armijo with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")

xopt, fopt, it, xIter, ls_iter = steepest_descent(rosenbrock, x0, 'wolfe', c1, c2, alpha0)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Wolfe with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")

xopt, fopt, it, xIter, ls_iter = steepest_descent(rosenbrock, x0, 'armijo', c1, c2, alpha0, tol)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Armijo with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")


xopt, fopt, it, xIter, ls_iter = steepest_descent(rosenbrock, x0, 'wolfe', c1, c2, alpha0, tol)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Wolfe with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it,ls_iter))

Method                                           xopt                 fopt              iter           ls_iter    
Armijo with default tol = 0.01           [0.9892, 0.9784]          5.0420e+01           964             5156           


Wolfe with default tol = 0.01            [0.9894, 0.9790]          5.0420e+01           960             6101           


Armijo with tol = 0.0001                 [0.9999, 0.9998]          5.0420e+01           1434            7185           


Wolfe with tol = 0.0001                  [0.9999, 0.9998]          5.0420e+01           1292            7896           


### Newton's method

In [16]:
print ("{:<40} {:^20} {:^20} {:^15}".format('Method','xopt','fopt','iter'))

xopt, fopt, it, xIter = newton_descent(rosenbrock, x0)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it))
print("\n")

xopt, fopt, it, xIter = newton_descent(rosenbrock, x0, tol=0.0001)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it))

Method                                           xopt                 fopt              iter      
Newton with default tol = 0.01           [1.0000, 1.0000]          5.0420e+01           3              


Newton with tol = 0.0001                 [1.0000, 1.0000]          5.0420e+01           3              


## Simple polynomial function
Here, a simple polynomial function is considered. 

In [ ]:
x0 = np.array([1.0])

xopt, fopt, it, xIter, ls_iter = steepest_descent(polynomial, x0, 'armijo', c1, c2, alpha0)
print ("{:<40} [{:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Steepest descent with default tol = 0.01',xopt[0],' ',fopt,it))

xopt, fopt, it, xIter = newton_descent(polynomial, x0, stepRule='armijo')
print ("{:<40} [{:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with Armijo default tol = 0.01',xopt[0],' ',fopt,it))

xopt, fopt, it, xIter = newton_descent(polynomial, x0)
print ("{:<40} [{:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with default tol = 0.01',xopt[0],' ',fopt,it))



Steepest descent with default tol = 0.01 [0.0018]          2.0833e+00           9              
Newton with Armijo default tol = 0.01    [0.0025]          2.0833e+00           596            
